In [1]:
# import the necessary packages
import csv
import h5py
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model, load_model 
from imutils import paths
import numpy as np
import cv2
from keras.layers.core import*
from feature_preprocess import feature_vector
from keras import layers

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load the model
resnet = load_model('models/ResNet50_trained_f3d.h5')
feature = load_model('models/feature_network_rename.h5')
model_resnet = Model(inputs=resnet.input, outputs=resnet.get_layer('flatten').output)
model_feature = Model(inputs=feature.input, outputs=feature.get_layer('f_dropout2').output)

merged_model = layers.concatenate([model_resnet.output, model_feature.output])
final_output = Dense(18, activation='softmax', name='shopee_output')(merged_model)

model = Model(inputs=[model_resnet.input, model_feature.input], outputs=final_output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [3]:
model.load_weights('models/bottleneck.h5', by_name=True)

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='combined_model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [10]:
# resnet50 with feature network
answer = [0 for i in range(16111)]
imagepaths = list(paths.list_images('images/Test'))
# pre-process the image for classification
count = 0
for imagepath in imagepaths:
    count = count + 1
    number = int(imagepath.split('.')[0][17:])
    
    img = load_img(imagepath, target_size=(200, 200))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image = preprocess_input(x)
    assert image.shape == (1,200,200,3)
    
    gray = cv2.imread(imagepath, 0)
    feature = feature_vector(gray)
    feature = np.expand_dims(feature, axis=0)
    assert feature.shape == (1,2014)
    
    problist = model.predict([image, feature])[0]
    maxIndex = np.argmax(problist)
    
    if count%100 == 0:
        print(count)
        
    answer[number - 1] = maxIndex


printlist = [[str(i + 1), str(answer[i])] for i in range(16111)]

with open('result3.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "category"])
    writer.writerows(printlist)

/usr/local/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100


In [ ]:
# resnet50 pre trained model
answer = [0 for i in range(16111)]
imagepaths = list(paths.list_images('images/Test'))
# pre-process the image for classification
count = 0
for imagepath in imagepaths:
    count = count + 1
    number = int(imagepath.split('.')[0][17:])
    img = load_img(imagepath, target_size=(200, 200))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image = preprocess_input(x)
    
    assert image.shape == (1,200,200,3)
    
    problist = model.predict(image)[0]
    maxIndex = np.argmax(problist)
            
    answer[number - 1] = maxIndex


printlist = [[str(i + 1), str(answer[i])] for i in range(16111)]

with open('result2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "category"])
    writer.writerows(printlist)

In [ ]:
# first try using resnet18
def processMono(path, outsize=64):

    post = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    post_size = post.shape

    if post_size[0] > post_size[1]:
        post = cv2.copyMakeBorder(post, 0, 0, int((post_size[0] - post_size[1]) / 2), int((post_size[0] - post_size[1]) / 2), cv2.BORDER_CONSTANT, value=255)
    if post_size[0] < post_size[1]:
        post = cv2.copyMakeBorder(post, int((post_size[1]-post_size[0])/2), int((post_size[1]-post_size[0])/2), 0, 0, cv2.BORDER_CONSTANT, value=255)

    post = cv2.resize(post, (outsize, outsize), interpolation=cv2.INTER_AREA)
    return post

In [ ]:
answer = [0 for i in range(16111)]
imagepaths = list(paths.list_images('images/Test'))
# pre-process the image for classification
count = 0
for imagepath in imagepaths:
    count = count + 1
    number = int(imagepath.split('.')[0][17:])
    image = processMono(imagepath)
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    
    assert image.shape == (1,64,64,1)
    
    problist = model.predict(image)[0]
    maxIndex = np.argmax(problist)
            
    answer[number - 1] = maxIndex


printlist = [[str(i + 1), str(answer[i])] for i in range(16111)]

with open('result.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id", "category"])
    writer.writerows(printlist)

In [ ]:
# following is for understanding of saved model
model = h5py.File('models/ResNet50_trained_f2a.h5','r')

In [ ]:
def printname(name):
    print(name)

In [ ]:
model.visit(printname)

In [ ]:
def load_label(name):
    label_dataset = h5py.File(name, "r")
    label = np.array(label_dataset["label"][:])
    return label

In [ ]:
img_labels = load_label('label_img_train.h5')
fea_labels = load_label('label_feature_train.h5')